In [2]:
from helpers import *
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
truth_seeker = pd.read_csv("TruthSeeker2023/Features_For_Traditional_ML_Techniques.csv")
truth_seeker = truth_seeker.drop(columns=["Unnamed: 0"])

In [4]:
truth_seeker_content = truth_seeker[["unique_count", "present_verbs", "total_count", "past_verbs", "ORG_percentage", "adjectives", 
                                     "NORP_percentage", "pronouns", "GPE_percentage", "TOs", "PERSON_percentage", "determiners", 
                                     "MONEY_percentage", "conjunctions", "DATE_percentage", "dots", "CARDINAL_percentage", "exclamation", 
                                     "PERCENT_percentage", "questions", "ORDINAL_percentage", "ampersand", "FAC_percentage", "capitals", 
                                     "LAW_percentage", "quotes", "PRODUCT_percentage", "digits", "EVENT_percentage", "long_word_freq", 
                                     "TIME_percentage", "short_word_freq", "LOC_percentage", "ORG_percentage", "WORK_OF_ART_percentage", 
                                     "QUANTITY_percentage", "LANGUAGE_percentage", "Max word length", "Min word length", "Average word length"]]

truth_seeker_user = truth_seeker[['followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'listed_count', 
                                  'following', 'mentions', 'quotes','replies', 'retweets', 'favourites', 'hashtags', 'URLs']]

truth_seeker_cred = truth_seeker[['BotScore', 'BotScoreBinary', 'cred', 'normalize_influence']]

truth_seeker_lexical = truth_seeker[['present_verbs', 'past_verbs', 'adjectives', 'adverbs', 'adpositions', 'pronouns', 'TOs', 
                                     'determiners', 'conjunctions', 'dots', 'exclamation', 'questions', 'ampersand', 'capitals', 
                                     'digits', 'long_word_freq', 'short_word_freq']]

truth_seeker_spaCy = truth_seeker[["ORG_percentage", "NORP_percentage", "GPE_percentage", "PERSON_percentage", "MONEY_percentage", "DATE_percentage", 
                                   "CARDINAL_percentage", "PERCENT_percentage", "ORDINAL_percentage", "FAC_percentage", "LAW_percentage", 
                                   "PRODUCT_percentage", "EVENT_percentage", "TIME_percentage", "LOC_percentage", "ORG_percentage", 
                                   "WORK_OF_ART_percentage", "QUANTITY_percentage", "LANGUAGE_percentage"]]

truth_seeker_features = truth_seeker.drop(columns=["majority_target", "statement", "BinaryNumTarget", "tweet", "embeddings"])

truth_seeker_output = truth_seeker["BinaryNumTarget"]

In [5]:
rf_model = RandomForestClassifier()
train_and_evaluate_data(truth_seeker_features, truth_seeker_output, rf_model)

Model Accuracy: 0.6877198211624441
Precision: 0.7070102589154861
Recall: 0.6709161499681289
F1 Score: 0.6884904706686885


In [6]:
train_and_evaluate_data(truth_seeker_spaCy, truth_seeker_output, rf_model)

Model Accuracy: 0.6054545454545455
Precision: 0.6268619035597072
Recall: 0.5755345656834907
F1 Score: 0.6001027159300323


In [7]:
train_and_evaluate_data(truth_seeker_lexical, truth_seeker_output, rf_model)

Model Accuracy: 0.6269448584202683
Precision: 0.637444193193019
Recall: 0.6370748102219389
F1 Score: 0.6372594481799212


In [8]:
train_and_evaluate_data(truth_seeker_cred, truth_seeker_output, rf_model)

Model Accuracy: 0.620059612518629
Precision: 0.6304071246819338
Recall: 0.6316856927623573
F1 Score: 0.6310457610929404


In [9]:
train_and_evaluate_data(truth_seeker_user, truth_seeker_output, rf_model)

Model Accuracy: 0.6402682563338301
Precision: 0.6615394196039357
Recall: 0.6155762878831779
F1 Score: 0.6377307519135524


In [10]:
train_and_evaluate_data(truth_seeker_content, truth_seeker_output, rf_model)

Model Accuracy: 0.6845305514157973
Precision: 0.6944914019236375
Recall: 0.6903865098221011
F1 Score: 0.692432872253865
